In [ ]:
# %%capture
!pip install unsloth vllm
!pip install --upgrade pillow
!pip install datasets
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b
!pip install tensorboard

import sys
modules = list(sys.modules.keys())
for x in modules:
    if "PIL" in x or "google" in x:
        sys.modules.pop(x)


In [ ]:

from unsloth import FastLanguageModel, PatchFastRL
import torch
PatchFastRL("GRPO", FastLanguageModel)

max_seq_length = 1024
lora_rank = 64

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="google/gemma-2-2b-it",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    fast_inference=False,
    max_lora_rank=lora_rank,
    gpu_memory_utilization=0.5,
)

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r=lora_rank,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=lora_rank,
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)


Unsloth 2025.8.1 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


In [ ]:
# Load and preprocess the PokerBench dataset.

from datasets import load_dataset, Dataset
import re

# System instruction to be included in the user prompt
SYSTEM_INSTRUCTION = """You are an expert poker player. Your response must be in the format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>"""

def inspect_dataset():
    """Inspect the PokerBench dataset to understand its structure."""
    data = load_dataset("RZ412/PokerBench")["train"]
    sample = data[0]
    print("Sample PokerBench record:", sample)
    print("Available keys:", list(sample.keys()))
    return data

def format_poker_prompt(game_state: dict) -> list:
    """Construct zero-shot prompt from game state, compatible with Gemma-2-2B."""
    # Fallback values for missing keys
    position = game_state.get('position', 'Unknown')
    stack = game_state.get('stack', 'Unknown')
    hand = game_state.get('hand', 'Unknown')
    community_cards = game_state.get('community_cards', 'None')
    pot = game_state.get('pot', 'Unknown')
    to_call = game_state.get('to_call', '0')
    opponent_actions = game_state.get('opponent_actions', 'None')

    # Check for alternative field names (adjust based on inspection)
    if 'player_position' in game_state:
        position = game_state['player_position']
    if 'stack_size' in game_state:
        stack = game_state['stack_size']
    if 'cards' in game_state:
        hand = game_state['cards']
    if 'board' in game_state:
        community_cards = game_state['board']
    if 'pot_size' in game_state:
        pot = game_state['pot_size']
    if 'amount_to_call' in game_state:
        to_call = game_state['amount_to_call']
    if 'actions' in game_state:
        opponent_actions = game_state['actions']

    prompt = f"""{SYSTEM_INSTRUCTION}

Game State:
- Position: {position}
- Stack: {stack}
- Hand: {hand}
- Community Cards: {community_cards}
- Pot: {pot}
- To Call: {to_call}
- Opponent Actions: {opponent_actions}

What action should you take (fold, call, raise, check)? Provide reasoning and final action."""
    return [
        {"role": "user", "content": prompt}
    ]

def extract_xml_answer(text: str) -> str:
    """Extract answer from XML-formatted response."""
    try:
        answer = text.split("<answer>")[-1].split("</answer>")[0].strip()
        return answer
    except:
        return ""

def get_pokerbench_dataset(split="train") -> Dataset:
    """Load and preprocess PokerBench dataset."""
    data = load_dataset("RZ412/PokerBench")[split]

    # Inspect the dataset to confirm structure
    print("Inspecting PokerBench dataset...")
    sample = data[0]
    print("Sample record:", sample)
    print("Available keys:", list(sample.keys()))

    # Test the chat template with a sample prompt
    print("Testing chat template...")
    sample_prompt = format_poker_prompt(sample)
    try:
        test_output = tokenizer.apply_chat_template(sample_prompt, tokenize=False, add_generation_prompt=True)
        print("Sample formatted prompt:", test_output)
    except Exception as e:
        print(f"Chat template error: {e}")
        raise

    # Map the dataset to create prompts
    try:
        data = data.map(
            lambda x: {
                "prompt": format_poker_prompt(x),
                "answer": x.get("action", "")  # Use .get to avoid KeyError
            },
            batched=False  # Process one example at a time to isolate errors
        )
    except Exception as e:
        print(f"Error during dataset mapping: {e}")
        raise
    return data

# Load and preprocess dataset
dataset = get_pokerbench_dataset()

In [5]:


# Reward Functions
# Define reward functions for GRPO training.

def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]["content"] for completion in completions]
    extracted = [extract_xml_answer(r) for r in responses]
    return [2.0 if r.lower() == a.lower() else 0.0 for r, a in zip(extracted, answer)]

def valid_action_reward_func(completions, **kwargs) -> list[float]:
    valid_actions = {"fold", "call", "raise", "check"}
    responses = [completion[0]["content"] for completion in completions]
    extracted = [extract_xml_answer(r).lower() for r in responses]
    return [0.5 if r in valid_actions else 0.0 for r in extracted]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    return [0.5 if re.match(pattern, r) else 0.0 for r in responses]

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    def count_xml(text: str) -> float:
        count = 0.0
        if text.count("<reasoning>\n") == 1:
            count += 0.125
        if text.count("\n</reasoning>\n") == 1:
            count += 0.125
        if text.count("\n<answer>\n") == 1:
            count += 0.125
        if text.count("\n</answer>") == 1:
            count += 0.125
        return count
    responses = [completion[0]["content"] for completion in completions]
    return [count_xml(r) for r in responses]

In [ ]:
# ## 5. Training with GRPO
# Configure and run GRPO training.

from trl import GRPOConfig, GRPOTrainer
from unsloth import is_bfloat16_supported

training_args = GRPOConfig(
    use_vllm=False,
    learning_rate=5e-6,
    adam_beta1=0.9,
    adam_beta2=0.99,
    weight_decay=0.1,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    optim="adamw_8bit",
    logging_steps=1,
    bf16=is_bfloat16_supported(),
    fp16=not is_bfloat16_supported(),
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    num_generations=4,
    max_prompt_length=256,
    max_completion_length=200,
    max_steps=1,
    save_steps=1,
    max_grad_norm=0.1,
    report_to="tensorboard",
    output_dir="outputs",
    run_name="gemma_pokerbench",
)

trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        xmlcount_reward_func,
        strict_format_reward_func,
        valid_action_reward_func,
        correctness_reward_func,
    ],
    args=training_args,
    train_dataset=dataset,
)

trainer.train()

In [7]:
from google.colab import drive
drive.mount('/content/drive')
model.save_pretrained("/content/drive/MyDrive/lora_adapter")

Mounted at /content/drive


In [8]:

from peft import PeftModel
from unsloth import FastLanguageModel

# Sample test prompt (adjust keys based on dataset schema from inspection)
test_prompt = format_poker_prompt({
    "player_position": "Button",
    "stack_size": 1000,
    "cards": ["As", "Kd"],
    "board": [],
    "pot_size": 100,
    "amount_to_call": 50,
    "actions": ["Small Blind posts 25", "Big Blind posts 50"]
})

# Tokenize the prompt
text = tokenizer.apply_chat_template(test_prompt, tokenize=False, add_generation_prompt=True)
inputs = tokenizer([text], return_tensors="pt", padding=True, truncation=True, max_length=1024).to("cuda")

# Debug input shape
print("Input shape:", inputs["input_ids"].shape)

# Inference without GRPO
print("Inference without GRPO:")
outputs = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_new_tokens=1024,
    temperature=0.8,
    top_p=0.95,
    do_sample=True,
)
output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output)


Input shape: torch.Size([1, 140])
Inference without GRPO:
user
You are an expert poker player. Your response must be in the format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>

Game State:
- Position: Button
- Stack: 1000
- Hand: ['As', 'Kd']
- Community Cards: []
- Pot: 100
- To Call: 50
- Opponent Actions: ['Small Blind posts 25', 'Big Blind posts 50']

What action should you take (fold, call, raise, check)? Provide reasoning and final action.
model
<reasoning>
This is a tricky spot.  The "Button" position is advantageous, but you're starting with a very weak hand, and the pot is relatively small.  We're dealing with a mix of value and potential for bluffing. 

* **Weak hand:** A-K is pretty marginal, often not strong enough to win in most post-flop situations.
* **Early position:**  Button is good but we're also at a low stack, meaning we need to be cautious.
* **Pot:**  The pot is small, but it's building quickly due to the blinds, so we want to make sure we're getting valu